In [156]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

%matplotlib inline

In [157]:
####### 160906_band2 ########
# START_FRAME=2
# END_FRAME=5517
# SEQ_NAME='160906_band2_with_ground'
# CONFIG_FILE='/home/lei/rajrup/KinectStream/Multiview/config/panoptic_160906_band2.json'

####### 170307_dance5 ########
# START_FRAME=34
# END_FRAME=10000
# SEQ_NAME='170307_dance5_with_ground'
# CONFIG_FILE='/home/lei/rajrup/KinectStream/Multiview/config/panoptic_170307_dance5.json'

####### 170915_office1 ########
# START_FRAME=19
# END_FRAME=5600
# SEQ_NAME='170915_office1_with_ground'
# CONFIG_FILE='/home/lei/rajrup/KinectStream/Multiview/config/panoptic_170915_office1.json'

####### 160906_pizza1 ########
# START_FRAME=1
# END_FRAME=1413
# SEQ_NAME='160906_pizza1_with_ground'
# CONFIG_FILE='/home/lei/rajrup/KinectStream/Multiview/config/panoptic_160906_pizza1.json'

# ####### 170915_toddler4 ########
START_FRAME=1
END_FRAME=3800
SEQ_NAME='170915_toddler4_with_ground'
CONFIG_FILE='/home/lei/rajrup/KinectStream/Multiview/config/panoptic_170915_toddler4.json'

with open(CONFIG_FILE) as f:
    config_data = json.load(f)
LOG_ID = config_data['log_id']

In [158]:
# mm_type='tracep1'
mm_type='wifi-25'

with_stalls = True
# with_stalls = False

In [159]:
def calc_avg_pssim(df, start_frame, end_frame, offset_start = 200, offset_end=50, with_stalls=True):
    df = df.copy()
    df = df[df['Frame'] <= end_frame - offset_end]
    df = df[df['Frame'] >= start_frame + offset_start]

    pssim_geo = df['PSSIM_GEO_BA'].values
    pssim_color = df['PSSIM_COLOR_BA'].values
    
    num_stalls = 0
    num_render_frames = 0
    for frame_id in range(start_frame + offset_start, end_frame - offset_end + 1):
        if frame_id % 10 != 0:
            continue
        
        if frame_id in df['Frame'].values:
            num_render_frames += 1
        else:
            num_stalls += 1

    if with_stalls:
        num_frames = num_render_frames + num_stalls
    else:
        num_frames = num_render_frames

    stall_percent = 100.0 * num_stalls / (num_render_frames + num_stalls)

    avg_pssim_geo = np.sum(pssim_geo) / num_frames
    avg_pssim_color = np.sum(pssim_color) / num_frames
    return avg_pssim_geo, avg_pssim_color, num_stalls, num_render_frames, num_frames, stall_percent

In [160]:
# DRACO
pipeline_method='draco'
culling='o3d_draco_s_pcull_c_cull'

pssim_file_path = f'/datassd/pipeline_ptcl/client_ptcl/{pipeline_method}/3D_pssim_{SEQ_NAME}_{pipeline_method}_{culling}_{mm_type}_log{LOG_ID}.csv'
draco_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
draco_pssim_df.fillna(0, inplace=True)

draco_pssim_df

,Frame,PSSIM_GEO_AB,AVG_PSSIM_GEO_AB,PSSIM_GEO_BA,AVG_PSSIM_GEO_BA,PSSIM_COLOR_AB,AVG_PSSIM_COLOR_AB,PSSIM_COLOR_BA,AVG_PSSIM_COLOR_BA
0,10,99.91,99.91,99.91,99.91,99.97,99.97,99.97,99.97
1,20,97.34,98.63,97.40,98.66,99.43,99.70,99.43,99.70
2,30,18.92,72.06,30.34,75.88,93.82,97.74,94.40,97.94
3,220,1.77,54.49,17.56,61.30,74.23,91.87,76.40,92.55
4,720,1.46,43.88,2.58,49.56,80.20,89.53,74.11,88.86
5,870,1.69,36.85,15.76,43.92,76.14,87.30,76.11,86.74
6,930,18.15,34.18,29.69,41.89,88.38,87.45,92.11,87.51
7,940,96.77,42.00,96.87,48.76,97.98,88.77,98.01,88.82
8,950,96.79,48.09,96.89,54.11,97.94,89.79,97.97,89.84
9,960,96.81,52.96,96.90,58.39,97.91,90.60,97.94,90.65


In [161]:
avg_pssim_geo, avg_pssim_color, num_stalls, num_render_frames, num_frames, stall_percent = calc_avg_pssim(draco_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls)
print(f'Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%, stall:  num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}')

Method:draco, avg_pssim_geo: 10.867295774647888, avg_pssim_color: 13.527915492957746, stall_percent:85.63380281690141%, stall:  num_stalls: 304, num_render_frames: 51, num_frames: 355


In [162]:
pipeline_method='starline++'
culling='o3d_nvenc_s_nocull_c_cull'

pssim_file_path = f'/datassd/pipeline/client_tiled/{pipeline_method}/3D_pssim_{SEQ_NAME}_{pipeline_method}_{culling}_{mm_type}_log{LOG_ID}.csv'
starline_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
starline_pssim_df.fillna(0, inplace=True)

starline_pssim_df

,Frame,PSSIM_GEO_AB,AVG_PSSIM_GEO_AB,PSSIM_GEO_BA,AVG_PSSIM_GEO_BA,PSSIM_COLOR_AB,AVG_PSSIM_COLOR_AB,PSSIM_COLOR_BA,AVG_PSSIM_COLOR_BA
0,70,90.70,90.70,89.12,89.12,86.62,86.62,85.17,85.17
1,80,89.09,89.90,88.99,89.06,86.92,86.77,86.89,86.03
2,90,89.11,89.64,88.99,89.03,86.81,86.79,86.78,86.28
3,100,88.36,89.32,88.24,88.83,87.00,86.84,86.94,86.44
4,110,87.81,89.02,87.61,88.59,87.25,86.92,87.20,86.60
...,...,...,...,...,...,...,...,...,...
352,3730,87.53,86.94,87.35,84.84,86.41,87.31,86.14,85.27
353,3740,87.47,86.94,87.31,84.85,86.51,87.31,86.38,85.27
354,3750,87.77,86.94,87.62,84.86,86.55,87.31,86.35,85.28
355,3760,87.26,86.94,87.07,84.86,86.30,87.31,86.22,85.28


In [163]:
avg_pssim_geo, avg_pssim_color, num_stalls, num_render_frames, num_frames, stall_percent = calc_avg_pssim(starline_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls)
print(f'Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%, num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}')

Method:starline++, avg_pssim_geo: 81.4010704225352, avg_pssim_color: 81.88177464788733, stall_percent:3.943661971830986%, num_stalls: 14, num_render_frames: 341, num_frames: 355


In [164]:
pipeline_method='livo'
culling='o3d_nvenc_s_kpcull10_c_cull'

pssim_file_path = f'/datassd/pipeline/client_tiled/{pipeline_method}/3D_pssim_{SEQ_NAME}_{pipeline_method}_{culling}_{mm_type}_log{LOG_ID}.csv'
livo_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
livo_pssim_df.fillna(0, inplace=True)

livo_pssim_df

,Frame,PSSIM_GEO_AB,AVG_PSSIM_GEO_AB,PSSIM_GEO_BA,AVG_PSSIM_GEO_BA,PSSIM_COLOR_AB,AVG_PSSIM_COLOR_AB,PSSIM_COLOR_BA,AVG_PSSIM_COLOR_BA
0,40,88.37,88.37,90.50,90.50,87.40,87.40,87.55,87.55
1,70,92.25,90.31,91.45,90.97,87.34,87.37,86.64,87.10
2,80,90.18,90.27,92.08,91.34,86.97,87.23,87.46,87.22
3,90,91.44,90.56,91.41,91.36,87.43,87.28,87.46,87.28
4,100,90.65,90.58,90.64,91.21,87.16,87.26,87.12,87.24
...,...,...,...,...,...,...,...,...,...
341,3740,87.71,88.53,87.55,88.26,87.64,87.86,87.58,87.19
342,3750,87.58,88.53,87.44,88.26,87.56,87.85,87.51,87.19
343,3760,87.72,88.53,87.53,88.26,87.33,87.85,87.30,87.19
344,3770,87.57,88.52,87.41,88.26,87.27,87.85,87.18,87.19


In [165]:
avg_pssim_geo, avg_pssim_color, num_stalls, num_render_frames, num_frames, stall_percent = calc_avg_pssim(livo_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls)
print(f'Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%,  num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}')

Method:livo, avg_pssim_geo: 81.98881690140844, avg_pssim_color: 81.07684507042254, stall_percent:7.042253521126761%,  num_stalls: 25, num_render_frames: 330, num_frames: 355


In [166]:
pipeline_method='starline'
culling='o3d_nvenc_s_nocull_c_cull'

pssim_file_path = f'/datassd/pipeline/client_tiled/{pipeline_method}/3D_pssim_{SEQ_NAME}_{pipeline_method}_{culling}_{mm_type}_log{LOG_ID}.csv'
starline_only_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
starline_only_pssim_df.fillna(0, inplace=True)

starline_only_pssim_df

,Frame,PSSIM_GEO_AB,AVG_PSSIM_GEO_AB,PSSIM_GEO_BA,AVG_PSSIM_GEO_BA,PSSIM_COLOR_AB,AVG_PSSIM_COLOR_AB,PSSIM_COLOR_BA,AVG_PSSIM_COLOR_BA
0,40,86.86,86.86,83.59,83.59,87.53,87.53,84.50,84.50
1,80,86.84,86.85,86.56,85.08,86.91,87.22,86.87,85.68
2,90,86.87,86.86,86.55,85.57,86.91,87.12,86.92,86.10
3,100,86.86,86.86,86.58,85.82,87.30,87.16,87.23,86.38
4,110,86.91,86.87,86.61,85.98,87.58,87.25,87.58,86.62
...,...,...,...,...,...,...,...,...,...
361,3740,86.54,86.75,86.35,84.93,87.44,88.27,87.33,86.60
362,3750,86.60,86.75,86.36,84.93,87.38,88.27,87.28,86.60
363,3760,86.54,86.75,86.25,84.94,87.08,88.27,86.98,86.60
364,3770,86.58,86.75,86.36,84.94,87.02,88.26,86.92,86.60


In [167]:
avg_pssim_geo, avg_pssim_color, num_stalls, num_render_frames, num_frames, stall_percent = calc_avg_pssim(starline_only_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls)
print(f'Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%,  num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}')

Method:starline, avg_pssim_geo: 83.46402816901409, avg_pssim_color: 85.13799999999999, stall_percent:1.6901408450704225%,  num_stalls: 6, num_render_frames: 349, num_frames: 355
